# Premikanje robota v notranjih koordinatah

1. Seznanite se z robotom. Napotke najdete v beležki *Prvi koraki.ipynb*. Pojdite skozi posamezne celice do konca.

2. Premikajte robota po sklepih relativno in absolutno.

3. Premikajte robota v kartezičnih koordinatah relativno in absolutno.

4. Naredite robota podajnega in zajemite trajektorijo v prostoru sklepov na 100 Hz.

5. Naredite robota ponovno togega - na varen način.

6. Izvedite posneto trajektorijo v prostoru sklepov na pribl. 100 Hz. Uporabite metodo `robot.GoTo_q` v zanki. **Bodite pozorni, da skoki v referenčnih vrednostih niso preveliki!**

7. Izriši zajeto in ponovno izvedeno trajektorijo.


In [1]:
import rospy
import inspect
from robotblockset.ros.grippers_ros import PandaGripper
from importlib import reload
import example_data_recorder
reload(example_data_recorder)
from utils import SoftSetJointCompliance
ns = "pingvin_2"
rospy.init_node(ns)
from robotblockset.ros.franka import panda_ros
r = panda_ros(ns=ns, control_strategy="JointImpedance", init_node=False)
g = PandaGripper(namespace=ns,robot=r)

Make sure that ROS node is initialized outside
Panda::Initialized
Panda:Gripper:ROS:Created


In [2]:
r.ErrorRecovery()

0

In [ ]:
r.CMoveFor([0.02,0.05,-0.1], 3)

In [2]:
r.CMove([0.1, 0, 0.3], 4)

0

In [ ]:
r.JMoveFor([-0.2,0,0,0,0,0,0],3)

In [ ]:
g.Close()

In [ ]:
g.Open()

In [ ]:
r.SetJointCompliant()

In [ ]:
r.ResetCurrentTarget()
SoftSetJointCompliance(r,r._franka_default.JointCompliance.K,4)

In [ ]:
!rostopic list | grep F_ext

In [ ]:
!rostopic echo -n 1 /pingvin_2/franka_state_controller/F_ext

In [ ]:
!rostopic echo -n 1 /pingvin_2/franka_state_controller/F_ext/wrench

In [ ]:
!rosbag record /pingvin_2/franka_state_controller/joint_states

In [ ]:
r._semaphore.release()

In [ ]:
import time
from robotblockset.transformations import rot_x, rp2t, q2r
from robotblockset.robots import isrobot
from robotblockset.graphics import plotjtraj, plotctraj
import matplotlib.pyplot as plt
import numpy as np

# Primer strukture za shranjevanje stanja robota se nahaja v datoteki example_data_recorder.py
from example_data_recorder import RobotDataRecorder

# Nastavimo parametre za snemanje in iniciliziramo medpomnilnik
recorder = RobotDataRecorder(r,max_iterations=5000)
r.SetCaptureCallback(recorder.record_callback)

# Definramo želeno lego
p0 = [0.5, 0, 0.5]
R0 = q2r(rot_x(np.pi))
T0 = rp2t(R0,p0)

r.ResetCurrentTarget() # Nastavi referenčno konfiguracijo na trenutno stanje  
r.ErrorRecovery() # Poskrbimo, da je robot v pravilnem stanju
r.SetJointStiff() # Nastavi ojačanja v kontrolerju

In [ ]:
# Zaženemo snemanje
r.SetJointStiff()
r.StartCapture()
# Sprožimo premik - POZOR, robot se bo premaknil!
r.CMove(T0, 5)
# Končamo snemanje
r.StopCapture()

In [ ]:
# Prikaz na grafu, polne črte so dejansko, črtkane pa želeno stanje

last_entry=recorder.it

fig, ax = plt.subplots(3, 1, figsize=(9, 6))
fig.suptitle("Joint trajectories")
plotjtraj(recorder.tt[:last_entry], recorder.qt[:last_entry], recorder.qdt[:last_entry], ax=ax)
plotjtraj(recorder.tt[:last_entry], recorder.rqt[:last_entry], recorder.rqdt[:last_entry], ax=ax, linestyle="--")

fig, ax = plt.subplots(3, 2, figsize=(9, 6))
fig.suptitle("Cartesiean trajectories")
plotctraj(recorder.tt[:last_entry], recorder.xt[:last_entry], recorder.vt[:last_entry], ax=ax)
plotctraj(recorder.tt[:last_entry], recorder.rxt[:last_entry], recorder.rvt[:last_entry], ax=ax, linestyle="--")

plt.show()

In [ ]:
#For kinestetic movement

import numpy as np
def record_robot_state(robot, frequency, duration):
    interval = 1.0 / frequency
    
    N=int(frequency*duration)
    tt = np.zeros((N,1))
    qt = np.zeros((N,r.nj))
    dqt = np.zeros((N,r.nj))

    i = 0
    start_time = time.monotonic()
    end_time = start_time + duration
    
    last_update = time.monotonic()

    while time.monotonic() <= end_time+interval and i < N:
        t = time.monotonic()
        if t - last_update >= interval:
            
            # Get robot state
            state = robot.GetState()
            
            tt[i] = t-start_time
            qt[i] = r.q
            dqt[i] = r.qdot

            # Update the last recorded time
            last_update = t
            i += 1
        
    return tt, qt, dqt, i, N   

In [ ]:
#Robot naj bo podajen
r.SetJointCompliant()

In [ ]:
import time
from utils import pip

time.sleep(1)
pip();
# vzorčimo 3 sekunde pri 100Hz
tt, qt, dqt, i, N = record_robot_state(r,100,3)
pip();

In [ ]:
from robotblockset.graphics import plotjtraj
import matplotlib.pyplot as plt

fig, ax = plt.subplots(3, 1, figsize=(9, 6))
fig.suptitle('Joint trajectories')
plotjtraj(tt, qt, dqt, ax=ax);
plotjtraj(tt, qt+0.01, dqt, ax=ax, linestyle="--");

In [ ]:
r.GoTo_q()